<a href="https://colab.research.google.com/github/SatrioHSamudra/Predator-Acoustic-Soundscape/blob/main/SharkAcoustic_SurfPerch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Leveraging AI to Unpick Predator-Prey Relationship in Coral Reefs**

# Installing the perch codebase from GitHub

In [ ]:
# As Colab updates its environment, the versions of the packages that are
# installed by default change. This can lead to compatibility issues with the
# perch package. We resolve this by: i) uninstalling some troublsome packages
# which now come as defaults with Colab; ii) installing the Perch package
# from a specific commit; iii) forcing the correct versions of some packages
# to ensure compatibility with one another.

# First: drop packages added by more recent colab env updates which cause errors
!pip uninstall -y jax-cuda12-pjrt jax-cuda12-plugin

# Second: install the pinned version of the surfperch repo from the specific commit.
!pip install git+https://github.com/google-research/perch.git@373253f5887e2964a51f348e107889dadfdcece0

# Third: Force-reinstall some troublesome packages to ensure compatibility
!pip install --force-reinstall jax==0.4.33 jaxlib==0.4.33 tensorflow==2.15 numpy==1.24.0

# Hosting the Data in Google Drive

In [ ]:
# The necessary pipeline to connect this Colab environment with the Google Drive
# folder where we host the data used in this tutorial.
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Create the folder to store the sample data
drive_sample_data_folder = '/content/drive/MyDrive/SurfPerch Demo with Calling in Our Corals Data/'
if not os.path.exists(drive_sample_data_folder):
  os.mkdir(drive_sample_data_folder)

# This is the location that this tutorial will use to save data.
drive_output_directory = '/content/drive/MyDrive/SurfPerch Demo with Calling in Our Corals Outputs/'
if not os.path.exists(drive_output_directory):
  os.mkdir(drive_output_directory)

# Copy the Data to Google Drive

In [ ]:
# Copy the sample data from GCS to Google Drive. If you are accidentally running this again, it is safe to interupt this cell and move on.
print('Copying data to Google Drive...')
!gsutil -m rsync -r gs://chirp-public-bucket/surfperch/ "/content/drive/MyDrive/SurfPerch Demo with Calling in Our Corals Data/" # Folder path reference 1

# Check the folders got copied. We need the SurfPerch-model, Australia, Indonesia, Philippines, and Seychelles folders.
!echo "Folders successfully copied:" && ls -l "/content/drive/MyDrive/SurfPerch Demo with Calling in Our Corals Data/" | grep "^d" # Folder path reference 2

# Setup and Configuration

In [ ]:
# Import various dependencies, including the relevant modules from the Perch
# repository. Note that "chirp" is the old name that the Perch team used, so any
# chirp modules imported here were installed as part of the Perch repository in
# one of the previous cells.

import collections
from collections import Counter
from etils import epath
from IPython.display import HTML
import ipywidgets as widgets
from IPython.display import display as ipy_display, HTML
import matplotlib.pyplot as plt
from ml_collections import config_dict
import numpy as np
import pandas as pd
from scipy.io import wavfile
import shutil
import tensorflow as tf
import tqdm

from chirp.inference import colab_utils
colab_utils.initialize(use_tf_gpu=True, disable_warnings=True)

from chirp import audio_utils
from chirp import config_utils
from chirp import path_utils
from chirp.inference import embed_lib
from chirp.inference import models
from chirp.inference import tf_examples
from chirp.models import metrics
from chirp.inference.search import bootstrap
from chirp.inference.search import search
from chirp.inference.search import display
from chirp.inference.classify import classify
from chirp.inference.classify import data_lib

# If connected to a Colab GPU runtime we should see a GPU listed
tf.config.list_physical_devices()

# Set the Configuration to Use

In [ ]:
# Model specific parameters: PLEASE DO NOT CHANGE THE CODE IN THIS CELL.
config = config_dict.ConfigDict()
embed_fn_config = config_dict.ConfigDict()
embed_fn_config.model_key = 'taxonomy_model_tf'
model_config = config_dict.ConfigDict()

# The size of each "chunk" of audio.
model_config.window_size_s = 5.0

# The hop size
model_config.hop_size_s = 5.0

# All audio in this tutorial is resampled to 32 kHz.
model_config.sample_rate = 32000

# The location of the pre-trained model.
model_config.model_path = drive_sample_data_folder + 'SurfPerch-model/'

# Only write embeddings to reduce size. The Perch codebase supports serializing
# a variety of metadata along with the embeddings, but for the purposes of this
# tutorial we will not need to make use of those features.
embed_fn_config.write_embeddings = True
embed_fn_config.write_logits = False
embed_fn_config.write_separated_audio = False
embed_fn_config.write_raw_audio = False

config.embed_fn_config = embed_fn_config
embed_fn_config.model_config = model_config

# These two settings can be used to break large inputs up into smaller chunks;
# this is especially helpful for dealing with long files or very large datasets.
# Given free colab has limited resources, you may want to reduce shard_len_s to
# 10 to prevent system RAM from becoming overloaded.
config.shard_len_s = 60 #
config.num_shards_per_file = -1

# Number of parent directories to include in the filename. This allows us to
# process raw audio that lives in multiple directories.
config.embed_fn_config.file_id_depth = 1

# If your dataset is large its useful to split the TFRecords across multiple
# shards so I/O operations can be parallized.
config.tf_record_shards = 10

# Specify the data (inputs) and results (outputs) directories

In [ ]:
# @title Hit run on this cell and choose a dataset
# Custom CSS to increase label width
style = """
<style>
.widget-label {
    min-width: 100px !important;
}
</style>
"""

# Apply the CSS
ipy_display(HTML(style))

# Define the dropdown
dropdown = widgets.Dropdown(
    options=['Indonesia', 'Australia','Philippines','Seychelles'],
    value='Seychelles',
    description='Current choice:',
    disabled=False,
)

# Define a function that reacts to changes in the dropdown
def on_dataset_change(change):
    dataset_choice = change['new']
    print(f'Changed dataset to: {dataset_choice}. Now work through the cells below for this dataset.')

# Attach the observer to the dropdown
dropdown.observe(on_dataset_change, names='value')

# Display the dropdown
ipy_display(dropdown)

In [ ]:
# Get the chosen datasets name
dataset_folder = dropdown.value + '/'

# Specify a glob pattern matching any number of wave files.
# Use [wW][aA][vV] to match .wav or .WAV files
unlabeled_audio_pattern = os.path.join(drive_sample_data_folder, dataset_folder, 'raw_audio/*.[wW][aA][vV]')

# Specify a directory where the embeddings will be written.
embedding_output_dir = os.path.join(drive_output_directory, dataset_folder, 'raw_embeddings/')
if not os.path.exists(embedding_output_dir):
  os.makedirs(embedding_output_dir, exist_ok=True)

config.output_dir = embedding_output_dir
config.source_file_patterns = [unlabeled_audio_pattern]

# Create output directory and write the configuration.
output_dir = epath.Path(config.output_dir)
output_dir.mkdir(exist_ok=True, parents=True)

# Don't forget to run the dropdown cell above!

## Write the configuration to JSON to ensure consistency with later stages of the pipeline

In [ ]:
# This dumps a config json file next to the embeddings that allows us to reuse
# the same embeddings and ensure that we have the correct config that was used
# to generate them.
embed_lib.maybe_write_config(config, output_dir)

# Create SourceInfos configuration, used in sharded computation when computing
# embeddings. These source_infos contain metadata about how we're going to
# partition the search corpus.  In particular, we're splitting the Powdermill
# audio into hundreds of 5s chunks, and the source_infos help us keep track of
# which chunk came from which raw audio file.
source_infos = embed_lib.create_source_infos(
    config.source_file_patterns,
    config.num_shards_per_file,
    config.shard_len_s)
print(f'Constructed {len(source_infos)} source infos.')

## Load the Pre-trained Embedding Model

In [ ]:
#@title { vertical-output: true }
# Here we're loading our generic Bird Classifier model.
# The embed_fn object is a wrapper over the model.
embed_fn = embed_lib.EmbedFn(**config.embed_fn_config)
print('\n\nLoading model(s)...')
embed_fn.setup()

print('\n\nTest-run of model...')
z = np.zeros([int(model_config.sample_rate * model_config.window_size_s)])
embed_fn.embedding_model.embed(z)
print('Setup complete!')

# Generate Embeddings

## Process the search dataset

In [ ]:
# To reduce the overhead computational resources required and speed up execution
# time, we use multiple threads to load the audio before embedding. This tends
# to perform faster, but can fail if any audio files are corrupt.

# The source_infos variable contains metadata about how to parition the search
# corpus.  This step creates an audio_iterator which iterates over the 5 second
# chunks of audio.

embed_fn.min_audio_s = 1.0
record_file = (output_dir / 'embeddings.tfrecord').as_posix()
succ, fail = 0, 0

audio_loader = lambda fp, offset: audio_utils.load_audio_window(
    fp, offset, model_config.sample_rate, config.shard_len_s)
audio_iterator = audio_utils.multi_load_audio_window(
    audio_loader=audio_loader,
    filepaths=[s.filepath for s in source_infos],
    offsets=[s.shard_num * s.shard_len_s for s in source_infos],
)

## Embed the search dataset

In [ ]:
#@title { vertical-output: true }
# Embed! This step may take several minutes to run.
with tf_examples.EmbeddingsTFRecordMultiWriter(
    output_dir=output_dir, num_files=config.tf_record_shards) as file_writer:
  for source_info, audio in tqdm.tqdm(
      zip(source_infos, audio_iterator), total=len(source_infos)):
    if audio.shape[0] < embed_fn.min_audio_s * model_config.sample_rate:
      # Ignore short audio.
      continue
    file_id = source_info.file_id(config.embed_fn_config.file_id_depth)
    offset_s = source_info.shard_num * source_info.shard_len_s
    example = embed_fn.audio_to_example(file_id, offset_s, audio)
    if example is None:
      fail += 1
      continue
    file_writer.write(example.SerializeToString())
    succ += 1
  file_writer.flush()
print(f'\n\nSuccessfully processed {succ} source_infos, failed {fail} times.')

fns = [fn for fn in output_dir.glob('embeddings-*')]
ds = tf.data.TFRecordDataset(fns)
parser = tf_examples.get_example_parser()
ds = ds.map(parser)
for ex in ds.as_numpy_iterator():
  print('Recording filename:', ex['filename'])
  print('Shape of the embedding:', ex['embedding'].shape)
  break

# This can take a few moments to get started

# Audio similarity search

### View the sound types from Seychelles dataset

In [ ]:
#@title { vertical-output: true }
# Path to cioc target sound folders
cioc_sounds = drive_sample_data_folder + dataset_folder + 'cioc_sounds'
cioc_sounds_folders = os.listdir(cioc_sounds)

# For each target sound folder, find the first audio file as an example
example_target_sounds = []
for folder in cioc_sounds_folders:
  wav_files = [file for file in os.listdir(os.path.join(cioc_sounds, folder)) if file.lower().endswith('.wav')]
  example_sound_path = os.path.join(cioc_sounds, folder + '/' + wav_files[0])
  example_target_sounds.append(example_sound_path)

# Now view each example target sound
print('Number of different target sounds: ', len(example_target_sounds))
for audio_path in example_target_sounds:
  print('Target sound label: ', audio_path.split('/')[-2])
  audio = audio_utils.load_audio(audio_path, model_config.sample_rate)
  display.plot_audio_melspec(audio, model_config.sample_rate)

### Pick a sound type

In [ ]:
# @title Hit run on this cell and pick a target sound

# Ensure the path exists and list directories
if os.path.exists(cioc_sounds):
    sound_folders = [f for f in os.listdir(cioc_sounds) if os.path.isdir(os.path.join(cioc_sounds, f))]
else:
    print("Path does not exist:", cioc_sounds)
    sound_folders = []

# Create and display the dropdown
sound_dropdown = widgets.Dropdown(
    options=sound_folders,
    description='Select sound:',
    disabled=False,
)

# Define a function that reacts to changes in the dropdown
def on_sound_change(change):
    choice = change['new']
    print(f'Changed target sound to: {choice}. Now work through the cells below for this dataset.')

# Attach the observer to the dropdown
sound_dropdown.observe(on_sound_change, names='value')

ipy_display(sound_dropdown)

### Load and view the query audio sample

In [ ]:
#@title Load and view the query audio sample { vertical-output: true }
target_sound = sound_dropdown.value
target_classes = [target_sound, 'Unknown']

# Select one of the target audio files. Default with 1, but for many sounds
# CIOC users surfaced multiple copies which we can leverage. The print
# out from this cell will tell you if there are others to choose from.
file_index = 1  #@param
file_index = file_index -1

# Build the folder path
target_audio_folder = os.path.join(drive_sample_data_folder, dataset_folder, 'cioc_sounds', target_sound)

# Retrieve all .wav files
wav_files = [file for file in os.listdir(target_audio_folder) if file.lower().endswith('.wav')]

# Print the total number of audio files
print(f"Number of indexed audio files in target sound directory: {len(wav_files)}")

# Validate the user input and select the audio file
if 0 <= file_index < len(wav_files):
    audio_path = os.path.join(target_audio_folder, wav_files[file_index])
    print(f"Viewing example: {file_index + 1}")
else:
    print("Invalid file index. Please select a valid index up to and including: ", len(wav_files))
    audio_path = None

# Assuming the rest of the code executes only if a valid path is selected
if audio_path:
    audio = audio_utils.load_audio(audio_path, model_config.sample_rate)
    display.plot_audio_melspec(audio, model_config.sample_rate)

## Select the specific query window

In [ ]:
#@title { vertical-output: true }
# If you're audio clip is longer than 5s, adjust start_s to pick your
# prefered start time.
start_s = 0  #@param

# Display the selected window.
print('Selected audio window:')
st = int(start_s * model_config.sample_rate)
end = int(st + model_config.window_size_s * model_config.sample_rate)
if end > audio.shape[0]:
  end = audio.shape[0]
  st = max([0, int(end - model_config.window_size_s * model_config.sample_rate)])
audio_window = audio[st:end]
display.plot_audio_melspec(audio_window, model_config.sample_rate)

query_audio = audio_window
sep_outputs = None

## Copy CIOC samples to labeled data directory

In [ ]:
# The path to an empty directory where the generated labeled samples will be
# placed. Each labeled sample will be placed into a subdirectory corresponding
# to the target class that we select for that sample.
target_audio_outputs = os.path.join(drive_output_directory, dataset_folder, target_sound + '/labeled_outputs/' + target_sound)
os.makedirs(target_audio_outputs, exist_ok=True)

# Copy all .wav and .WAV files from target_audio_folder to labeled_data_path
for file in os.listdir(target_audio_folder):
    if file.lower().endswith('.wav'):
        source_path = os.path.join(target_audio_folder, file)
        destination_path = os.path.join(target_audio_outputs, file)
        shutil.copy2(source_path, destination_path)

### Embed the query

In [ ]:
query = query_audio

embedded_query = embed_fn.embedding_model.embed(query).embeddings[ :, 0, :]

 ### Create a TensorFlow Dataset (TFDS) wrapper over the embeddings

In [ ]:
# Use the embedded dataset that we created above...
bootstrap_config = bootstrap.BootstrapConfig.load_from_embedding_path(
    embeddings_path=embedding_output_dir,
    annotated_path=target_audio_outputs
)

project_state = bootstrap.BootstrapState(
    bootstrap_config, embedding_model=embed_fn.embedding_model)

embeddings_ds = project_state.create_embeddings_dataset()

## Run top-k search using a comparison metric

### Agile modelling step 1

In [ ]:
# Number of search results to capture. top_k = 25 is often a good start,
# but we use 10 for brevity in this demo.
top_k = 25 # @param {type:"number"}

# The Perch codebase supports:
#  'euclidean', which is the standard euclidean distance
#  'cosine', which is the cosine similarity,
#  'mip', which is Maximum Inner Product
metric = 'euclidean'  #@param['euclidean', 'mip', 'cosine']

# Target distance for search results. This lets us try to hone in on a
# 'classifier boundary' instead of just looking at the closest matches.
# Set to 'None' for raw 'best results' search.
target_score = None #@param

results, all_scores = search.search_embeddings_parallel(
    embeddings_ds, embedded_query,
    hop_size_s=model_config.hop_size_s,
    top_k=top_k, target_score=target_score, score_fn=metric,
    random_sample=False)

### User-in-the-loop data labeling

The cell below displays the search results in a user-interface in the following format:
* Image: a plot visualization of the audio search result (Mel spectrogram, frequency in Hz over time)
* A playback of the audio sample itself
* Metrics and metadata: `rank` position, `source file` of the recording segment, `offset_s` (in seconds) from the recording, and the search `score` (i.e. similarity with the query)
* Candidate labels for the sample

**Instructions to the user:** <br>
For each search result presented below, select either the target label or Unknown class. Samples which present difficult cases can be highly valuable to label, but if any cannot be labeled with confidence then its possible to leave them blank.

**Quick guide on how to verify vocalisations:**

One of the hardest parts of the entire process can be learning to correctly identify the target sounds for yourself. Often this requires a skilled bioacoustician to do well. Many common reef sounds can blur from one clear signal into another, especially tricky pulse, purr and pop vocalizations. When in doubt, take the time to check back against your original target sound. Make use of listening with a good set of earphones or headphones and also visually assessing the spectrogram.

For an easy start try the whoop of the Ambon Damselfish in the Australian dataset!

In [ ]:
#@title Display the search results for user labeling. { vertical-output: true }
display.display_search_results(
    project_state=project_state,
    results=results,
    embedding_sample_rate=model_config.sample_rate,
    checkbox_labels=target_classes,
    max_workers=5)

# Let this cell finish executing before labeling the samples

### Choose a target score by plotting a histogram of all distances in the search corpus

During the top-k search step above, the Perch code also computed and saved the distances to *every* point in the search corpus. The actual numerical values of these distances are hard to interpret, but the relative values are very useful. In the following cell we plot a histogram of this set of distances to help us conceptualize the geometry of our embedded dataset. This histogram helps us find and tune our values for the `target_score` variable in the top-k search.

A typical histogram will appear to fit some vaguely-normal looking distribution, possibly skewed left with a heavy tail. While there is no prescriptive formula for finding useful values of `target_score`, the Perch team has found that good choices tend to lie near the left-hand 'hockey-stick' point of the distribution. For example, in the following histogram, you might try playing with values somewhere in the range of 2.8 to 3.1:

<img src="https://drive.google.com/uc?id=1bLc2XDTqutihg4wJSkCfB2DiT4Dpr4UO" width="40%" height="40%">

These tend to correspond to examples that are faint, or have background noise, or are otherwise not especially obvious. Annotating these examples and adding them to the training set is very important because they help the linear model discriminate better on these less-clear "boundary" points. Though take care to make sure your annotations are correct, if in doubt you can leave samples unlabeled.

In [ ]:
#@title { vertical-output: true }
# Plot histogram of distances.
ys, _, _ = plt.hist(all_scores, bins=128, density=True)
hit_scores = [r.score for r in results.search_results]
plt.scatter(hit_scores, np.zeros_like(hit_scores), marker='|',
            color='r', alpha=0.5)

plt.xlabel(metric)
plt.ylabel('density')
if target_score is not None:
  plt.plot([target_score, target_score], [0.0, np.max(ys)], 'r:')
  # Compute the proportion of scores < target_score.
  hit_percentage = (all_scores < target_score).mean()
  print(f'score < target_score percentage : {hit_percentage:5.3f}')
min_score = np.min(all_scores)
plt.plot([min_score, min_score], [0.0, np.max(ys)], 'g:')

plt.show()

### Write the user-annotated search results to file

This cell saves the annotations you generated in the previous step to your Google Drives.  It writes data to the `labeled_data_path` location that was specified above.

In [ ]:
def write_labeled_data(search_results, labeled_data_path: str, sample_rate: int):
  """Write labeled results to the labeled data collection."""
  labeled_data_path = epath.Path(labeled_data_path)
  counts = collections.defaultdict(int)
  duplicates = collections.defaultdict(int)
  for r in search_results:
    labels = [ch.description for ch in r.label_widgets if ch.value]
    if not labels:
      continue
    extension = epath.Path(r.filename).suffix
    filename = epath.Path(r.filename).name[: -len(extension)]
    output_filename = f'{filename}___{r.timestamp_offset}{extension}'
    for label in labels:
      output_path = labeled_data_path / label
      output_path.mkdir(parents=True, exist_ok=True)
      output_filepath = epath.Path(output_path / output_filename)
      if output_filepath.exists():
        duplicates[f'{label}'] += 1
        continue
      else:
        counts[label] += 1
      with output_filepath.open('wb') as f:
        wavfile.write(f, sample_rate, np.float32(r.audio))
  for label, count in counts.items():
    print(f'Wrote {count} examples for label {label}')
  for label, count in duplicates.items():
    print(f'Not saving {count} duplicates for label {label}')

labeled_data_path = os.path.join(drive_output_directory, dataset_folder, target_sound + '/labeled_outputs/')
write_labeled_data(results, labeled_data_path, model_config.sample_rate)

<a name=active-learning></a>
## Train a Linear Classifier with active learning 🤖🧠

#### Agile modelling step: 2


In the agile modelling step 1 above, we labeled samples from the search dataset as matches (by similarity comparison with the queries) for our target classes. If you're satisfied you labelled enough samples during this step, then from here on you will likely want to focus your efforts using the active agile modelling step 2 cells below.

We will now train a simple linear model using those bootstrapped (labeled) data points from the search dataset.

**Important:** in order to be able to train the linear model, we need several examples from each of the two classes (more is better!). If you encounter an error in this section when training the model, you likely did not generate a sufficient amount of labeled data. Please go back to the [Top-k Search](#top-k-search) section and choose a new value of the `target_score` attribute and label some more data.

In [ ]:
# @title Load and embed the search-annotated dataset { vertical-output: true }

# Load the training data that is located in the `labeled_data_path` directory.
# In that directory there will be folders corresponding to our target labels

merged = data_lib.MergedDataset.from_folder_of_folders(
    base_dir=labeled_data_path,
    embedding_model=project_state.embedding_model,
    time_pooling='mean',
    load_audio=False,
    target_sample_rate=-2,
    audio_file_pattern='*',
    embedding_config_hash=bootstrap_config.embedding_config_hash(),
)

# Label distribution
lbl_counts = np.sum(merged.data['label_hot'], axis=0)
print('num classes :', (lbl_counts > 0).sum())
print('mean ex / class :', lbl_counts.sum() / (lbl_counts > 0).sum())
print('min ex / class :', (lbl_counts + (lbl_counts == 0) * 1e6).min())

### Train a simple linear model using the labeled embeddings

We use the following hyperparameters which should serve as reasonably performing defaults to train this linear model (classifier):

- `batch_size`: 12
- `num_epochs`: 128
- `num_hiddens`: -1 (to match the dimensions of the embeddings)
- `learning_rate`: 0.001

Additionally, we compute the following metrics to measure the "goodness" of the trained model:
- `acc`: overall accuracy
- `auc_roc`: AUC ROC, or area under the receiving curve
- `cmAP`: mean average precision averaged across species
- `maps`: mean average precision for each class

In [ ]:
# Number of random training examples to choose from each class.

# Note that if you don't have very many samples you'll need to set
# train_ratio=None and train_examples_per_class to a value that is
# less than the minimum number of examples you have of each class.

# Set exactly one of train_ratio and train_examples_per_class
train_ratio = 0.8  #@param
train_examples_per_class = None  #@param

# Number of random re-trainings. In other words, this value indicates how many
# models we will train, each will use a new randomly selected combination of
# our labeled samples for training and testing. By training multiple models,
# we get a sense of model robustness. Here, we train 3, but feel free to
# increase it for added confidence in the model's performance.
num_seeds = 3  #@param

# Classifier training hyperparams.
# These should be good defaults.
batch_size = 12
num_epochs = 128
num_hiddens = -1
learning_rate = 1e-3

In [ ]:
# This cell trains the linear model(s) and outputs some summary statistics for
# each model. If you only have num_seeds = 1 then we'll only train a single
# model here.
metrics = collections.defaultdict(list)
for seed in tqdm.tqdm(range(num_seeds)):
  if num_hiddens > 0:
    model = classify.get_two_layer_model(
        num_hiddens, merged.embedding_dim, merged.num_classes)
  else:
    model = classify.get_linear_model(
        merged.embedding_dim, merged.num_classes)
  run_metrics = classify.train_embedding_model(
      model, merged, train_ratio, train_examples_per_class,
      num_epochs, seed, batch_size, learning_rate)
  metrics['acc'].append(run_metrics.top1_accuracy)
  metrics['auc_roc'].append(run_metrics.auc_roc)
  metrics['cmap'].append(run_metrics.cmap_value)
  metrics['maps'].append(run_metrics.class_maps)
  metrics['test_logits'].append(run_metrics.test_logits)

#### Compute the average metrics and print the model performance

In the previous cells, the `num_seeds` param controls how many times we train a model.  Each time we train a model there is some randomness in terms of which data points we choose from our labeled data, as well as some randomness in the model's initialization.  We can get a sense of how robust our classifier is by training multple times and looking at the summary statistics computed by the following cell.  A low `auc_roc` value (ie, less than 0.9 or so) probably indicates that we should generate some more training data.

**Note**: If you are new to labeling marine sounds, it can also be tricky to be sure you've labeled everything correctly.

In [ ]:
mean_acc = np.mean(metrics['acc'])
mean_auc = np.mean(metrics['auc_roc'])
mean_cmap = np.mean(metrics['cmap'])
# Merge the test_logits into a single array.
test_logits = {
    k: np.concatenate([logits[k] for logits in metrics['test_logits']])
    for k in metrics['test_logits'][0].keys()
}

print(f'acc:{mean_acc:5.2f}, auc_roc:{mean_auc:5.2f}, cmap:{mean_cmap}')
for lbl, auc in zip(merged.labels, run_metrics.class_maps):
  if np.isnan(auc):
    continue
  print(f'\n{lbl:8s}, auc_roc:{auc:5.2f}')
  colab_utils.prstats(f'test_logits({lbl})',
                      test_logits[merged.labels.index(lbl)])

If you have completed the active learning loop below a few times, and have a high auc_roc score (>0.95 or close to it), then move on to the 'Inference stage' cell below.

### The Active Learning Loop: Generating more training examples

### Generate new samples using logit scores

In [ ]:
# Choose the target class to work with.  This must be one of the values from
# the target_classes list above (target_sound or 'Unknown')

target_class = target_sound  #@param

# Choose a target logit. You can start by setting this to 'None' to get the
# highest-logit examples, which should reveal more of the target sound.
# Next, selecting 0.0 or the hockey stick bend should reveal samples the model
# currently finds difficult - these are very valuable to label.
target_logit = none  #@param

# Number of results to display.
num_results = 10  #@param

# Create the embeddings dataset.
target_class_idx = merged.labels.index(target_class)
results, all_logits = search.classifer_search_embeddings_parallel(
    embeddings_classifier=model,
    target_index=target_class_idx,
    embeddings_dataset=embeddings_ds,
    hop_size_s=model_config.hop_size_s,
    target_score=target_logit,
    top_k=num_results
)

In [ ]:
#@title { vertical-output: true }
# Plot the histogram of model logits.
_, ys, _ = plt.hist(all_logits, bins=128, density=True)
plt.xlabel(f'{target_class} logit')
plt.ylabel('density')
plt.plot([target_logit, target_logit], [0.0, np.max(ys)], 'r:')
plt.show()

In [ ]:
#@title Display results for the target label { vertical-output: true }

display_labels = merged.labels

extra_labels = []  #@param
for label in extra_labels:
  if label not in merged.labels:
    display_labels += (label,)
if 'Unknown' not in merged.labels:
  display_labels += ('Unknown',)

display.display_search_results(
    results=results,
    project_state=project_state,
    embedding_sample_rate=model_config.sample_rate,
    checkbox_labels=target_classes,
    max_workers=5)

#### Add selected results to the labeled data

As before, once we've annotated the examples from the previous cell, we'll save them in the `labeled_data_dir`.

In [ ]:
results.write_labeled_data(labeled_data_path, model_config.sample_rate)

Now return to the 'Agile modelling step: 2' cell and work down from here cell by cell once again, until you are satisified with the auc_roc score output by the 'Compute the average metrics' cell.

If you are satsified by the score, move on to the final stage below...

### Inference stage: write the trained model's classification results

Usually the purpose of creating a model in the first place is to bulk-process many hours of raw audio.  In this cell, we'll run our linear model over the entire search corpus.  The output will be a CSV containing the results.  

The `threshold` parameter is the minimum logit score that will get recorded (ie, samples wfor which the model had a low confidence are  omitted from the results to ensure high confidence in the final results).  You can tune this score to generate different output CSVs at different confidence scores. Alternatively, if you notice in your results csv that many periods are omitted but you don't want to introduce potential error by lowering the logit threshold, you can increase the strength of logit scores by carefully labeling more data and retraining your classifier using the active learning process.

In [ ]:
# { vertical-output: true }

threshold = 1.0  #@param
output_filepath = os.path.join(drive_output_directory, dataset_folder, target_sound + '/inference_'+ target_sound + '.csv')

def classify_batch(batch):
  """Classify a batch of embeddings."""
  emb = batch[tf_examples.EMBEDDING]
  emb_shape = tf.shape(emb)
  flat_emb = tf.reshape(emb, [-1, emb_shape[-1]])
  logits = model(flat_emb)
  logits = tf.reshape(
      logits, [emb_shape[0], emb_shape[1], tf.shape(logits)[-1]])
  # Take the maximum logit over channels.
  logits = tf.reduce_max(logits, axis=-2)
  batch['logits'] = logits
  return batch


inference_ds = embeddings_ds.map(
    classify_batch, num_parallel_calls=tf.data.AUTOTUNE
)
all_embeddings = []
all_logits = []

with open(output_filepath, 'w') as f:
  # Write column headers.
  headers = ['filename', 'timestamp_s', 'label', 'logit']
  f.write(', '.join(headers) + '\n')
  for ex in tqdm.tqdm(inference_ds.as_numpy_iterator()):
    all_embeddings.append(ex['embedding'])
    all_logits.append(ex['logits'])
    for t in range(ex['logits'].shape[0]):
      for i, label in enumerate(merged.labels):
        if ex['logits'][t, i] > threshold:
          offset = ex['timestamp_s'] + t * model_config.hop_size_s

          logit = '{:.2f}'.format(ex['logits'][t, i])
          row = [ex['filename'].decode('utf-8'),
                 '{:.2f}'.format(offset),
                 label, logit]
          f.write(', '.join(row) + '\n')

all_embeddings = np.concatenate(all_embeddings, axis=0)
all_logits = np.concatenate(all_logits, axis=0)
print('Saved results to: ', output_filepath)

<a name=results></a>
# Results .csv files

With the example datasets used in this demo, we can also compare the occurence of target sounds between the relevant habitat types the data was gathered from.

The csv results file contains a list of all filenames and their predicted label. Below is some custom code for the datasets used in this demo which reads the filenames present in the csv and produces plots.




In [ ]:
# Find the output folders for each target sound analysed so far
sound_folders = os.listdir(os.path.join(drive_output_directory, dataset_folder))
if 'raw_embeddings' in sound_folders:
    sound_folders.remove('raw_embeddings')

# Store results in dict
all_results_dict = {}
for sound in sound_folders:
  # Load results csv
  results_csv_path = os.path.join(drive_output_directory,
                                  dataset_folder,
                                  sound + '/inference_'+ sound + '.csv')
  results_df = pd.read_csv(results_csv_path)

  # Strip white space
  results_df.columns = results_df.columns.str.strip()
  results_df = results_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

  # Find the files with the target sound and store their habitat type
  target_detected = results_df[results_df['label'] == sound]
  source_files = list(target_detected['filename'])
  habitat_type = [item[20] for item in source_files] # our data has the habitat type denoted at the 20th character in the filename
  habitat_counts = Counter(habitat_type)
  habitat_counts_dict = dict(habitat_counts)
  all_results_dict[sound] = habitat_counts_dict